In [1]:
import os
from nipype.interfaces import afni as afni
import nipype.pipeline.engine as pe 
import nipype.interfaces.io as nio
import nipype.interfaces.utility as util

import warnings
import re
import sys
from nipype.interfaces.afni.base import AFNICommand, AFNICommandInputSpec
from nipype.interfaces.base import CommandLineInputSpec, CommandLine, TraitedSpec, traits, File, InputMultiPath, \
    isdefined
import glob

#from nipype.interfaces.afni.ccpafni import CCPDeconvolve
from interfaces import CCPDeconvolve, OneDTool, CCPMaskTool


data_dir = os.path.abspath('../DATA')
output_dir = data_dir + '/outputs_AFNI'
print data_dir
print os.listdir(data_dir)

# LOGGING TO FILE
from nipype import config, logging
config.update_config({'logging': {'workflow_level': 'DEBUG',
                                  'interface_level': 'DEBUG',
                                  'log_directory': data_dir,
                                  'log_to_file': True}})
logging.update_logging(config)

# DATA
subject_list = ['sub001', 'sub003'] 

print output_dir
tr_counts = [1000, 1000, 1000, 761]

/home/brain/nipype/ccp/DATA
['pypeline.log', 'wf1', 'sub003', 'sub001', 'union', 'ccpdeconvolve', 'mean', 'tcat', 'movdemean', 'scale', 'outputs_WF1', 'pypeline.lock', 'voxeldim']
/home/brain/nipype/ccp/DATA/outputs_AFNI


In [2]:
infosource = pe.Node(util.IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

In [3]:
# DATA SOURCE

templates = {'myfiles': data_dir + "/{subject_id}/run00[1,2].nii.gz",
             'movregs': data_dir + "/{subject_id}/Movement_Regressors_ALL.txt",
             'onsets': data_dir + "/{subject_id}/onsets_COND00[1,3]_ALL.txt"}

sf = pe.Node(interface=nio.SelectFiles(templates), 
             name='sf')
#sf.iterables = ('subject_id', subject_list)
#sf.inputs.base_directory = data_dir
sf.inputs.sort_filelist = True
#sf.inputs.force_lists = True
#sf.inputs.subject_id = 'sub001'
sf.inputs.force_lists = ['myfiles', 'onsets']
#sf.inputs.subject_id = 'subject_id'
sf.inputs




base_directory = <undefined>
force_lists = ['myfiles', 'onsets']
ignore_exception = False
raise_on_empty = True
sort_filelist = True
subject_id = <undefined>

In [4]:
wf_mov_magnitude = pe.Node(interface=OneDTool(),
                 name='wf_mov_magnitude')

#wf_mov_magnitude.base_dir = data_dir
#wf_mov_magnitude.inputs.in_file = movregs
wf_mov_magnitude.inputs.tr_counts = tr_counts
wf_mov_magnitude.inputs.out_file = 'mov_magnitude_enorm.1D'
wf_mov_magnitude.inputs.args = '-derivative'
wf_mov_magnitude.inputs.collapse_cols = 'euclidean_norm'
wf_mov_magnitude.inputs


args = -derivative
collapse_cols = euclidean_norm
environ = {}
ignore_exception = False
in_file = <undefined>
out_file = mov_magnitude_enorm.1D
terminal_output = stream
tr_counts = [1000, 1000, 1000, 761]

In [5]:
#DATA SINK
ds = pe.Node(interface=nio.DataSink(),
            name="ds")
ds.inputs.base_directory = output_dir
#ds.inputs.container = output_dir
ds.inputs.parameterization = False
substitutions = [('_subject_id_', '')]
ds.inputs.substitutions = substitutions

In [6]:
wf = pe.Workflow(name="wf")
wf.base_dir = data_dir

In [7]:
wf.connect(infosource, 'subject_id', sf, 'subject_id')
wf.connect(infosource, 'subject_id', ds, 'container')
wf.connect(sf, 'movregs', wf_mov_magnitude, 'in_file')
wf.connect(wf_mov_magnitude, 'out_file', ds, 'magnitude')
#wf.connect(wf_mov_magnitude, ds, [('out_file','magnitude')])

wf.connect(sf, 'onsets', ds, 'onsets')

DEBUG:workflow:(wf.infosource, wf.sf): No edge data
DEBUG:workflow:(wf.infosource, wf.sf): new edge data: {'connect': [('subject_id', 'subject_id')]}
DEBUG:workflow:(wf.infosource, wf.ds): No edge data
DEBUG:workflow:(wf.infosource, wf.ds): new edge data: {'connect': [('subject_id', 'container')]}
DEBUG:workflow:(wf.sf, wf.wf_mov_magnitude): No edge data
DEBUG:workflow:(wf.sf, wf.wf_mov_magnitude): new edge data: {'connect': [('movregs', 'in_file')]}
DEBUG:workflow:(wf.wf_mov_magnitude, wf.ds): No edge data
DEBUG:workflow:(wf.wf_mov_magnitude, wf.ds): new edge data: {'connect': [('out_file', 'magnitude')]}
DEBUG:workflow:(wf.sf, wf.ds): No edge data
DEBUG:workflow:(wf.sf, wf.ds): new edge data: {'connect': [('onsets', 'onsets')]}


In [8]:
wf.run()
wf.write_graph(graph2use='exec')

DEBUG:workflow:Creating flat graph for workflow: wf
DEBUG:workflow:expanding workflow: wf
DEBUG:workflow:processing node: wf.infosource
DEBUG:workflow:processing node: wf.sf
DEBUG:workflow:processing node: wf.wf_mov_magnitude
DEBUG:workflow:processing node: wf.ds
DEBUG:workflow:finished expanding workflow: wf
INFO:workflow:['check', 'execution', 'logging']
DEBUG:workflow:PE: expanding iterables
DEBUG:workflow:Detected iterable nodes [wf.infosource]
DEBUG:workflow:Expanding the iterable node wf.infosource...
DEBUG:workflow:node: wf.infosource iterables: {'subject_id': <function <lambda> at 0x7f6e2e540500>}
DEBUG:workflow:('subnodes:', [wf.infosource, wf.sf, wf.wf_mov_magnitude, wf.ds])
DEBUG:workflow:setting nodelevel(wf.infosource.aI) input subject_id = sub001
DEBUG:workflow:setting nodelevel(wf.infosource.aI) input subject_id = sub003
DEBUG:workflow:PE: expanding iterables ... done
DEBUG:workflow:setting nodelevel(wf.sf.a1) input subject_id = sub003
DEBUG:workflow:setting nodelevel(wf

In [10]:
from nipype.utils.filemanip import loadpkl
print loadpkl('../DATA/wf/_subject_id_sub001/wf_mov_magnitude/result_wf_mov_magnitude.pklz').runtime.merged

[]


In [29]:
wf_mov_magnitude.write_report(report_type='preexec', cwd=output_dir)

TypeError: 'NoneType' object has no attribute '__getitem__'

In [31]:
wf_mov_magnitude.output_dir()

'/tmp/tmpP5Ujku/wf/wf_mov_magnitude'

In [34]:
wf.base_dir

'/home/brain/nipype/ccp/DATA'

In [ ]:
#in function 'write_report' (pipeline/engine/nodes.py)

if op.exists(report_file):
    return report_file
# will return a path to the report file